In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
from torch.utils.data import Dataset, DataLoader
import glob
import wandb
import os
import torch.optim as optimizers
import dfs_code
from torch_geometric.data import InMemoryDataset, Data
import pickle
import torch
import torch.nn as nn
import tqdm
import copy
import pandas as pd
import torch.nn.functional as F
import sys
import yaml
import functools
from ml_collections import ConfigDict
sys.path = ['../../src'] + sys.path
from dfs_transformer import DFSCodeSeq2SeqFC, Deepchem2TorchGeometric, Trainer, to_cuda, Enzymes

In [3]:
def collate_fn(dlist, alpha=0, n_classes=384):
    node_batch = [] 
    edge_batch = []
    y_batch = []
    code_batch = []
    for d in dlist:
        node_batch += [d.node_features.clone()]
        edge_batch += [d.edge_features.clone()]
        code_batch += [d.min_dfs_code.clone()]
        y_batch += [d.y]
    y = torch.tensor(y_batch, dtype=torch.long)
    #y = (1-alpha)*y + alpha/n_classes
    return code_batch, node_batch, edge_batch, y

In [4]:
m = ConfigDict()
t = ConfigDict()
d = ConfigDict()

In [5]:
m["class"] = "DFSCodeSeq2SeqFC"
m["n_atoms"] = 26
m["n_bonds"] = 8
m["emb_dim"] = 60
m["nhead"] = 12
m["nlayers"] = 6
m["max_nodes"] = 200
m["max_edges"] = 600
m["dim_feedforward"] = 2048
m["missing_value"] = None
m["n_node_features"] = 26
m["n_edge_features"] = 8
m["n_class_tokens"] = 1 
m["use_min"] = True

t["batch_size"] = 50
t["gpu_id"] = 0
t["load_last"] = False
t["fingerprint"] = "cls"
t["accumulate_grads"] = 2
t["alpha"] = 0.0
t["clip_gradient"] = 0.5
t["decay_factor"] = 0.8
t["es_improvement"] = 0.0
t["es_path"] = None
t["es_patience"] = 10
t["es_period"] = 166
t["lr_head"] = 0.003
t["lr_encoder"] = 0.0003 # 0.00003
t["lr_patience"] = 3
t["lr_adjustment_period"] = 166
t["wdecay_encoder"] = 0.0
t["n_epochs"] = 25
t["struct"] = True
t["seed"] = 123
t["num_workers"] = 8

d["n_classes"] = 384
d["path"] = "/mnt/ssd/datasets/enzyme/restrictive_n200_dleq4.5.pkl"
d["n_edge_types"] = 8
d["n_node_types"] = 122

In [6]:
mode = "online"
name = "dfstr*.-%d-%d"%(m.emb_dim*5, m.nhead)
project = "enzymes-n200"

In [7]:
dataset = Enzymes()

In [8]:
train_idx = torch.tensor([idx for idx, d in enumerate(dataset) if d.split == "train"], dtype=torch.long)
valid_idx = torch.tensor([idx for idx, d in enumerate(dataset) if d.split == "valid"], dtype=torch.long)
test_idx = torch.tensor([idx for idx, d in enumerate(dataset) if d.split == "test"], dtype=torch.long)

In [9]:
coll_train = functools.partial(collate_fn, alpha=t.alpha, n_classes=d.n_classes)

In [10]:
trainloader = DataLoader(dataset, sampler=torch.utils.data.SubsetRandomSampler(train_idx), 
                         batch_size=t.batch_size, collate_fn=coll_train, num_workers=t.num_workers)
validloader = DataLoader(dataset, sampler=torch.utils.data.SubsetRandomSampler(valid_idx), 
                         batch_size=t.batch_size, collate_fn=collate_fn, num_workers=t.num_workers)
testloader = DataLoader(dataset, sampler=torch.utils.data.SubsetRandomSampler(test_idx), 
                        batch_size=t.batch_size, collate_fn=collate_fn, num_workers=t.num_workers)

In [11]:
config = ConfigDict()
config["model"] = m
config["training"] = t
config["data"] = d

In [12]:
run = wandb.init(mode=mode, project=project, entity="dfstransformer", 
                 name=name, config=config.to_dict(), job_type="evaluation")

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: chrisxx (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.3 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
2021-10-01 19:07:01.731774: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/opt/cuda/extras/CUPTI/lib64/:/opt/intel/lib:/opt/intel/mkl/lib/intel64:/opt/intel:/opt/ibm/ILOG/CPLEX_Studio1210/cplex/bin/x86-64_linux:/opt/ibm/ILOG/CPLEX_Studio1210/cplex/python/3.7/x86-64_linux:/opt/intel/clck_latest/lib:/opt/intel/daal/lib:/opt/intel/intelpython3/lib:/opt/intel/ipp/lib:/opt/intel/itac_2019/lib:/opt/intel/itac_latest/lib:/opt/intel/mkl/lib:/opt/intel/mkl_/lib:/opt/intel/mpirt

In [13]:
ce = nn.CrossEntropyLoss(ignore_index=-1)

In [14]:
class TransformerPlusHead(nn.Module):
    def __init__(self, encoder, n_classes, fingerprint='cls'):
        super(TransformerPlusHead, self).__init__()
        self.encoder = encoder
        n_encoding = encoder.get_n_encoding(fingerprint)
        self.head = nn.Linear(n_encoding, n_classes)
        self.fingerprint = fingerprint
    
    def forward(self, C, N, E):
        features = self.encoder.encode(C, N, E, method=self.fingerprint)
        output = self.head(features)
        return output
        

In [15]:
def loss(pred, y, ce=ce):
    return ce(pred, y)

def acc(pred, y):
    return torch.sum(torch.argmax(pred, dim=1) == y)/len(y)

In [16]:
device = torch.device('cuda:%d'%t.gpu_id if torch.cuda.is_available()  else 'cpu')
encoder = DFSCodeSeq2SeqFC(**m)
    
#if t.load_last and model_dir is not None:
#    encoder.load_state_dict(torch.load(model_dir+'/checkpoint.pt', map_location=device))

In [17]:
model = TransformerPlusHead(encoder, d.n_classes, fingerprint=t.fingerprint)

In [18]:
param_groups = [
    {'amsgrad': False,
     'betas': (0.9,0.98),
     'eps': 1e-09,
     'lr': t.lr_encoder,
     'params': model.encoder.parameters(),
     'weight_decay': t.wdecay_encoder},
    {'amsgrad': False,
     'betas': (0.9, 0.999),
     'eps': 1e-08,
     'lr': t.lr_head,
     'params': model.head.parameters(),
     'weight_decay': 0}
]

In [19]:
t

accumulate_grads: 2
alpha: 0.0
batch_size: 50
clip_gradient: 0.5
decay_factor: 0.8
es_improvement: 0.0
es_path: null
es_patience: 10
es_period: 166
fingerprint: cls
gpu_id: 0
load_last: false
lr_adjustment_period: 166
lr_encoder: 0.0003
lr_head: 0.003
lr_patience: 3
n_epochs: 25
num_workers: 8
seed: 123
struct: true
wdecay_encoder: 0.0

In [20]:
trainer = Trainer(model, trainloader, loss, validloader=validloader, metrics={'acc': acc}, wandb_run = run, param_groups=param_groups, **t)

In [21]:
trainer.fit()

Valid 3: loss 5.704285 0.0000:  11%|████████████████                                                                                                                                 | 1/9 [00:00<00:07,  1.14it/s]


Valid 5: loss 5.506699 0.0800:  11%|████████████████                                                                                                                                 | 1/9 [00:01<00:06,  1.21it/s]


Epoch 6: loss 2.682616 0.3810: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 166/166 [00:52<00:00,  3.16it/s]


EarlyStopping counter: 1 out of 10


Epoch 7: loss 2.464474 0.5000: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 166/166 [00:52<00:00,  3.18it/s]


...

EarlyStopping counter: 8 out of 10


Epoch 20: loss 0.143659 0.9762: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 166/166 [00:54<00:00,  3.07it/s]


EarlyStopping counter: 9 out of 10


Epoch 21: loss 0.134097 1.0000:  99%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏| 165/166 [00:54<00:00,  3.04it/s]

EarlyStopping counter: 10 out of 10


In [22]:
model.load_state_dict(torch.load(trainer.es_path+'checkpoint.pt'))

<All keys matched successfully>

In [23]:
def compute_acc(model, loader):
    with torch.no_grad():
        preds = []
        ys = []
        for data in tqdm.tqdm(loader):
            data = [to_cuda(dd, device) for dd in data]
            pred = model(*data[:-1])
            pred = torch.argmax(pred, dim=1).detach().cpu().numpy().tolist()
            y = data[-1].detach().cpu().numpy().tolist()
            preds += pred
            ys += y
    return (np.asarray(preds) == np.asarray(ys)).sum()/len(ys)
            

In [24]:
run.log({'Valid Accuracy': compute_acc(model, validloader)})
run.log({'Test Accuracy': compute_acc(model, testloader)})

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 36/36 [00:05<00:00,  6.28it/s]


In [25]:
compute_acc(model, testloader)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 36/36 [00:05<00:00,  6.71it/s]


0.2095505617977528

In [26]:
compute_acc(model, validloader)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:01<00:00,  5.93it/s]


0.21674876847290642